<a href="https://colab.research.google.com/github/D1105/Classifier/blob/main/pytorch_classification_LIGHTNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
!pip install pytorch-lightning -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 12.7 MB/s eta 0:00:00


In [18]:
import wandb
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

batch_size = 128
epochs = 10
learning_rate = 0.001

wandb.init(
    project="pytorch_classification_LIGHTNING",

    config={
    "learning_rate": learning_rate,
    "dataset": "CIFAR-10",
    "batch_size": batch_size,
    "epochs": epochs,
    }
)

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

random_seed = 42
torch.manual_seed(random_seed);

In [19]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pytorch_lightning as L
from torch.utils.data import DataLoader
from torch.utils.data import random_split



class CIFAR10DataModule(L.LightningDataModule):
    def __init__(self, data_dir, batch_size, num_workers):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
        )
    def prepare_data(self):
        datasets.CIFAR10(self.data_dir, train = True, download = True)
        datasets.CIFAR10(self.data_dir, train = False, download = True)
    def setup(self, stage):
        entire_dataset = datasets.CIFAR10(
            root = self.data_dir,
            train = True,
            transform = self.transform,
            download = False
        )
        self.train_ds, self.val_ds = random_split(entire_dataset, [45000, 5000])

        self.test_ds = datasets.CIFAR10(
            root = self.data_dir,
            train = False,
            transform = self.transform,
            download = False
        )
    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size = self.batch_size,
            num_workers = self.num_workers,
            shuffle = True
            )
    def val_dataloader(self):
        return DataLoader(
            self.val_ds,
            batch_size = 2*self.batch_size,
            num_workers = self.num_workers,
            shuffle = False
            )
    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size = self.batch_size,
            num_workers = self.num_workers,
            shuffle = False
            )


In [20]:
import torchmetrics
from typing import Any
from pytorch_lightning.utilities.types import STEP_OUTPUT, OptimizerLRScheduler
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import pytorch_lightning as L


class Net(L.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.save_hyperparameters()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128,kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256*4*4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task = 'multiclass', num_classes = num_classes)
        self.f1_score = torchmetrics.F1Score(task = 'multiclass', num_classes = num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))

        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))

        x = F.relu(self.conv5(x))
        x = self.pool(F.relu(self.conv6(x)))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        scores = self(x)
        loss = self.loss_fn(scores, y)
        accuracy = self.accuracy(scores, y)
        f1_score = self.f1_score(scores, y)
        self.log_dict({'train_loss':loss, 'train_accuracy':accuracy, 'train_f1_score':f1_score})
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        scores = self(x)
        loss = self.loss_fn(scores, y)
        self.log('val_loss',loss)
        return loss
    def test_step(self, batch, batch_idx):
        x, y = batch
        scores = self(x)
        loss = self.loss_fn(scores, y)
        accuracy = self.accuracy(scores, y)
        self.log('test_accuracy', accuracy)
        return loss
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr = learning_rate)


def save_Net(net, PATH):
    torch.save(net.state_dict(), PATH)


In [21]:
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(log_model="all")
net = Net(len(classes))
dm = CIFAR10DataModule(data_dir="dataset/", batch_size=batch_size, num_workers=4)
trainer = L.Trainer(logger = wandb_logger, accelerator = "gpu", devices = 1, min_epochs = 1, max_epochs = epochs)

trainer.fit(net, dm)

trainer.validate(net, dm)

trainer.test(net, dm)

wandb.finish()


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name     | Type               | Params
-------------------------------------------------
0  | conv1    | Conv2d             | 896   
1  | conv2    | Conv2d             | 18.5 K
2  | conv3    | Conv2d             | 73.9 K
3  | conv4    | Conv2d             | 147 K 
4  | conv5    | Conv2d             | 295 K 
5  | conv6    | Conv2d             | 590 K 
6  | pool     | MaxPool2d          | 0     
7  | fc1      | Linear             | 4.2 M 
8  | fc2      | Linear             | 524 K 
9  | fc3      | Linear             | 5.1 K 
10 | loss_fn  | CrossEntropyLoss   | 0     
11 | accuracy | MulticlassAccuracy | 0     
12 | f1_score | MulticlassF1Score  | 0     
-------------------------------------------------
5.9 M     Trainable params
0         Non-trainable params
5.9 M     Total params
23.405    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Files already downloaded and verified
Files already downloaded and verified


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7907000184059143     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇█
test_accuracy,▁
train_accuracy,▁▂▂▄▄▄▅▅▅▅▅▆▅▆▇▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇██▇█████
train_f1_score,▁▂▂▄▄▄▅▅▅▅▅▆▅▆▇▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇██▇█████
train_loss,█▇▆▆▅▅▄▅▄▄▄▃▄▃▂▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
val_loss,█▆▄▄▄▄▄▄▅▅▁
epoch,10
test_accuracy,0.7907
train_accuracy,0.92969
train_f1_score,0.92969
